# Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack.

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively.

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the
number of lesions with the diameter of the ROI to compensate different image sizes.

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc.

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation:
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed.

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [ ]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import csv
import io
import numpy as np
import math
from math import log2

In [ ]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [ ]:
def get_data(filename):
    data = []
    open_file = open(filename, 'r')

    while True:
      line = open_file.readline()
      if not line:
        break
      features = line.split(",")
      label = int(features[19])
      features = [float(x) for x in features[:19]]
      data.append(DataPoint(label, features))
     # print(data)
    return data


In [ ]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)

    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature '
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [ ]:
def make_prediction(tree_root, data_point):
  #do you have to traverse the tree?
#     your code goes here

    if tree_root.is_leaf == True:
      return tree_root.prediction

    #we go down the tree based on what?
    if data_point.features[tree_root.feature_idx] < tree_root.thresh_val:
      return make_prediction(tree_root.left_child, data_point)
    elif data_point.features[tree_root.feature_idx] >= tree_root.thresh_val:
      return make_prediction(tree_root.right_child, data_point)
    else:
      return None


Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [ ]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
    for dp in data:
      feature_value = float(dp.features[feature_idx])
      if feature_value < threshold:
          left_split.append(dp)
      else: right_split.append(dp)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [ ]:
def calc_entropy(data):
  sum = 0
  variable1 = 0
  variable2 = 0
  entrophy = 0
  for dp in data:
    #print(dp)
    temp_label = dp.label
    sum = sum + 1
    if temp_label == 1:
      variable1 = variable1 + 1
    else:
      variable2 = variable2 + 1
  if sum == 0:
    return 0
  if variable1 == 0:
    entropy = -(variable2/sum) * math.log2(variable2/sum)
  elif variable2 == 0:
      entropy = -(variable1/sum) * math.log2(variable1/sum)
  elif variable1 == 0 and variable2 == 0:
      return 0
  else:
    entropy = -(variable1/sum) * math.log2(variable1/sum) - (variable2/sum) * math.log2(variable2/sum)

#     your code goes here
  return entropy

In [ ]:
from numpy.lib.npyio import DataSource
data = get_data("messidor_features.csv")
calc_entropy(data)

0.9971705766292643

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [ ]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0
    best_thresh = None
    #calculate initial entrophy
    initial_entrophy = calc_entropy(data)
    #create clone

    #sort by feature value
    data.sort(key=lambda a: a.features[feature_idx])
    for i in range (0, len(data) - 1, 1):
        #calculates threshold value between every interval
        #0, 1, 18 binary set threshold immediately to .5
        #set gain
        first = float((data[i].features[feature_idx]))
        second = float((data[i + 1].features[feature_idx]))
        cur_thresh = (first + second)/2

        #splits data based on threshhold
        left_split, right_split = split_dataset(data, feature_idx, cur_thresh)
        left_len = len(left_split)
        right_len = len(right_split)

        #create leaf node if data split is empty?
        #if len(left_split) == 0 or len(right_split) == 0:
         # return(0, None)
      #gets two values
        entrophy1 = calc_entropy(left_split)
        entrophy2 = calc_entropy(right_split)
        temp_entrophy =  (((left_len)/len(data)) * (entrophy1)) + ((((right_len))/len(data)) * (entrophy2))
        temp_gain = initial_entrophy - temp_entrophy
        if temp_gain > best_info_gain:
          best_info_gain = temp_gain
          best_thresh = cur_thresh
        if feature_idx == 0 or feature_idx == 1 or feature_idx ==18:
          best_thresh = .5

    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [ ]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
    best_gain_ratio = 0

    for i in range (0, 19, 1):
      info_gain, thresh = calc_best_threshold(data, i)
      if thresh == None:
        thresh = 0
      left_split, right_split = split_dataset(data, i, thresh)

      var1 = len(left_split)
      var2 = len(right_split)
      #calculates split info
      if info_gain > best_gain_ratio:
        best_gain_ratio = info_gain
        best_thresh = thresh
        best_feature = i
        #don't know what to represent feature index as

    return (best_feature, best_thresh)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [ ]:
def create_leaf_node(data):
    yes = 0
    no = 0
    for dp in data:
      if dp.label == 1:
        yes = yes + 1
      else:
        no = no + 1

    mode = 1
    if no > yes:
      mode = 0
    node = TreeNode()
    node.is_leaf = True
    node.prediction = mode
    #print(node.prediction)
    #don't think prediction is correct
    return node

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [ ]:
from numpy.lib import roots

def create_decision_tree(data, max_levels):
    if max_levels == 1:
      node = create_leaf_node(data)
      return node

    best_feature_and_thresh = identify_best_split(data)
    if best_feature_and_thresh == (None, None):
        node = create_leaf_node(data)
        return node

    cur_node = TreeNode()
    cur_node.is_leaf = False
    cur_node.feature_idx = best_feature_and_thresh[0]
    cur_node.thresh_val = best_feature_and_thresh[1]
    #if len(data) == 0
    # best_feature, best_thresh = identify_best_split(data)
    left_split, right_split = split_dataset(data, cur_node.feature_idx, cur_node.thresh_val)

    cur_node.left_child = create_decision_tree(left_split, max_levels - 1)
    cur_node.right_child = create_decision_tree(right_split, max_levels - 1)

    return cur_node

#     your code goes here


Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [ ]:
def calc_accuracy(tree_root, data):
  accurate = 0
  print("treeroot", tree_root)
  for dp in data:
    #print(dp)
    prediction = make_prediction(tree_root, dp)
    #print(prediction)
    if prediction == dp.label:
      #print(prediction)
      #print(dp.label)
      accurate = accurate + 1
  calc_accuracy = float(accurate/(len(data)))
  return calc_accuracy

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [ ]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.csv")
 #create array of arrays
k_fold_start = 0
numfolds = 5
partition_length = int(len(d)/numfolds)
folds_array = []
folds_array.append(d[0:partition_length])
sum = 0


for i in range (1, numfolds, 1):
  folds_array.append(d[i*partition_length: (i+1)*partition_length])

for n in range (numfolds):
  train_set = []
  test_set = []
  for i in range (numfolds):
    if i == n:
      test_set = folds_array[i]
    else:
      train_set += folds_array[i]
  print ('Training set size:', len(train_set))
  print ('Test set size    :', len(test_set))
 # print ('Training set:', train_set)
  #calc_entropy(train_set)

  # the timer is just for fun! you will NOT be graded on runtime
  start = time.time()

  # create the decision tree
  tree = create_decision_tree(train_set, 10)

  end = time.time()
  print ('Time taken:', end - start)

  # calculate the accuracy of the tree
  accuracy = calc_accuracy(tree, test_set)
  sum += accuracy
  print ('The accuracy on the test set is ', str(accuracy * 100.0))
sum = sum/numfolds
print ('The avergae accuracy on the test set is ', str(sum * 100.0))






# train_val1 = fold1 +  fold2 +  fold3 + fold4
# test_val1 = fold5

# train_val2 = fold1 + fold2 + fold3 + fold5
# test_val2 = fold4

# train_val3 = fold1 + fold2 + fold4 + fold5
# test_val3 = fold3

# train_val4 = fold1 + fold3 + fold4 + fold5
# test_val4 = fold2

# train_val5 = fold2 + fold3 + fold4 + fold5
# test_val5 = fold1


# # partition data into train_set and test_set
# train_set = train_val1
# test_set = test_val1

# print ('Training set size:', len(train_set))
# print ('Test set size    :', len(test_set))

# the timer is just for fun! you will NOT be graded on runtime
# start = time.time()

# # create the decision tree
# tree = create_decision_tree(train_set, 10)

# end = time.time()
# print ('Time taken:', end - start)

# # calculate the accuracy of the tree
# accuracy = calc_accuracy(tree, test_set)
# print ('The accuracy on the test set is ', str(accuracy * 100.0))
# #t.printTree()

Training set size: 920
Test set size    : 230
Time taken: 24.773709058761597
treeroot <__main__.TreeNode object at 0x7f841805dd00>
The accuracy on the test set is  63.91304347826087
Training set size: 920
Test set size    : 230
Time taken: 23.579248189926147
treeroot <__main__.TreeNode object at 0x7f8415e04910>
The accuracy on the test set is  63.04347826086957
Training set size: 920
Test set size    : 230
Time taken: 21.52536678314209
treeroot <__main__.TreeNode object at 0x7f8418086b80>
The accuracy on the test set is  66.95652173913044
Training set size: 920
Test set size    : 230
Time taken: 23.275328636169434
treeroot <__main__.TreeNode object at 0x7f841812eb50>
The accuracy on the test set is  63.91304347826087
Training set size: 920
Test set size    : 230
Time taken: 22.96701431274414
treeroot <__main__.TreeNode object at 0x7f8415fe8880>
The accuracy on the test set is  64.34782608695652
The avergae accuracy on the test set is  64.43478260869566
